In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
from config.builders import build_chimera_config, build_dpcl_config, build_mi_config
import json
    
args = {
    'num_frequencies': 512,
    'num_mels': -1,
    'sample_rate': 44100,
    'hidden_size': 300,
    'bidirectional': True,
    'num_layers': 4,
    'embedding_size': 40,
    'num_sources': 2,
    'embedding_activation': ['tanh', 'unit_norm'],
    'mask_activation': ['softmax']
}

config = build_chimera_config(args)
with open('networks/chimera_config.json', 'w') as f:
    json.dump(config, f)

KeyError: 'trainable'

In [8]:
from networks import SeparationModel
from importlib import reload
import trainer
reload(trainer)
import numpy as np
import torch

loss_tuples = [('dpcl', 'embedding', 1.0)]
               #('mse', 'estimates', .5)]
t = trainer.Trainer(output_folder='test/', 
        train_data=None, 
        validation_data=None, 
        model='networks/chimera_config.json',
        loss_tuples=loss_tuples,
        options={
            'data_parallel': False,
            'device': 'cpu'
        })

device = t.device
model = t.module
test_data = np.random.random((1, 100, 512))
data = torch.from_numpy(test_data)
data = {'log_spectrogram': data,
         'magnitude_spectrogram': data}

a0 = torch.randint(0, 2, (1, 100, 512))
assignments = torch.stack([a0, 1-a0], dim=-1)
weights = torch.rand((1, 100, 512))
sources = torch.rand((1, 100, 512, 2))

target = {'assignments': assignments, 
          'weights': weights,
          'sources': sources}

def prepare_data(data, device):
    for k in data:
        data[k] = data[k].float().to(device)
    return data

data = prepare_data(data, device)
target = prepare_data(target, device)
output = model(data)
model.project_assignments(target['assignments'])

loss = t.calculate_loss(output, target)
sum(loss).backward()
print(loss)

[tensor(0.5240, grad_fn=<MulBackward>)]


In [41]:
embeddings

tensor([[[0.0439, 0.8477, 0.3971,  ..., 0.7753, 0.0547, 0.9786],
         [0.3782, 0.4074, 0.5107,  ..., 0.6275, 0.2279, 0.5542],
         [0.1048, 0.9835, 0.6676,  ..., 0.1544, 0.4599, 0.1064],
         ...,
         [0.5639, 0.0580, 0.0794,  ..., 0.6194, 0.2730, 0.7875],
         [0.2483, 0.6578, 0.8597,  ..., 0.1383, 0.2640, 0.3030],
         [0.0505, 0.2239, 0.3872,  ..., 0.6791, 0.7809, 0.4021]],

        [[0.1705, 0.9387, 0.1563,  ..., 0.5904, 0.1079, 0.3421],
         [0.3789, 0.4443, 0.0546,  ..., 0.0542, 0.8712, 0.7437],
         [0.3697, 0.4379, 0.9856,  ..., 0.5885, 0.7984, 0.2812],
         ...,
         [0.7819, 0.0265, 0.5804,  ..., 0.9745, 0.9026, 0.0726],
         [0.5433, 0.8991, 0.4388,  ..., 0.1924, 0.7758, 0.5085],
         [0.0142, 0.5743, 0.0376,  ..., 0.0436, 0.0764, 0.0613]],

        [[0.5929, 0.6472, 0.4401,  ..., 0.1365, 0.6311, 0.1979],
         [0.8342, 0.2901, 0.7416,  ..., 0.9982, 0.4906, 0.9263],
         [0.2811, 0.4352, 0.6739,  ..., 0.3135, 0.7237, 0.

In [9]:
output['embedding']

tensor([[[[ 2.4233e-01,  9.6219e-02,  1.6633e-01,  ..., -7.6923e-02,
            5.7023e-02, -1.1445e-01],
          [ 4.6402e-02,  1.7361e-01, -1.0590e-02,  ...,  1.3967e-01,
            1.2356e-01,  2.1546e-01],
          [-6.0788e-03, -1.0113e-01, -2.1805e-01,  ...,  3.4081e-02,
            1.2679e-01, -2.2962e-01],
          ...,
          [-1.1002e-01, -2.1847e-01,  2.9475e-01,  ...,  2.6439e-01,
            2.5229e-01,  8.9946e-02],
          [ 1.7057e-01, -4.7226e-02,  4.2949e-02,  ..., -4.6590e-02,
           -1.6946e-01, -1.4191e-01],
          [-2.4431e-01, -2.4302e-01,  6.7314e-02,  ...,  5.3848e-02,
           -1.2064e-01, -1.8532e-01]],

         [[ 1.9860e-01,  1.1002e-01,  1.6472e-01,  ..., -1.6407e-02,
            1.0030e-01, -1.5499e-01],
          [ 5.1155e-02,  1.6975e-01, -1.5114e-02,  ...,  1.6391e-01,
            9.3469e-02,  2.1145e-01],
          [ 3.2065e-03, -9.7714e-02, -2.0795e-01,  ...,  2.8785e-02,
            9.3135e-02, -1.9648e-01],
          ...,
     

In [10]:
def classic_deep_clustering(embeddings, assignments, weights):
    batch_size, num_points, embedding_size = embeddings.shape
    weights = weights.view(batch_size, num_points, 1)
    embeddings = weights.expand_as(embeddings) * embeddings
    assignments = weights.expand_as(assignments) * assignments
    # -- get normalization factor (normalize by sum over all applied weights)
    # See (7) from ALTERNATIVE OBJECTIVE FUNCTIONS FOR DEEP CLUSTERING, Wang et al, 2017
    # for how to apply weights.  Innermost square() because we're working with sqrt weights
    # Then use (\sum_{i} w_{ii})^2 = \sum_{ij} w_i * w_j and sum over batches
    count = torch.sum(torch.sum(weights**2, dim=1)**2)
    
    embeddings_transpose = embeddings.transpose(2, 1)
    assignments_transpose = assignments.transpose(2, 1)
    
    vTv = torch.matmul(embeddings_transpose, embeddings)
    vTy = torch.matmul(embeddings_transpose, assignments)
    yTy = torch.matmul(assignments_transpose, assignments)
    return (torch.sum(vTv**2) - 2*torch.sum(vTy**2) + torch.sum(yTy**2)) / count

In [13]:
import torch
from torch import nn

batch_size=1
num_points=100
for e in [2**i for i in range(0, 11)]:
    embedding_size=e
    embedding = torch.rand(batch_size, num_points, embedding_size)
    embedding = nn.functional.normalize(embedding, dim=-1, p=2)
    weights = torch.rand(batch_size, num_points)
    a0 = torch.randint(0,2, (batch_size, num_points))
    assignments = torch.stack([a0, 1-a0], dim=-1)

    output = {'embedding': embedding}
    target = {'weights': weights,
              'assignments': assignments}

    loss = t.calculate_loss(output, target)
    loss_cl = classic_deep_clustering(output['embedding'], target['assignments'], target['weights'])

    print('Embedding size: {:0>4d}\t New loss: {:.4f}, \tOld loss: {:.4f}'.format(e, sum(loss).item(), loss_cl.item()))

Embedding size: 0001	 New loss: 0.4997, 	Old loss: 0.4997
Embedding size: 0002	 New loss: 0.4015, 	Old loss: 0.4015
Embedding size: 0004	 New loss: 0.3515, 	Old loss: 0.3515
Embedding size: 0008	 New loss: 0.3321, 	Old loss: 0.3321
Embedding size: 0016	 New loss: 0.3078, 	Old loss: 0.3078
Embedding size: 0032	 New loss: 0.3092, 	Old loss: 0.3092
Embedding size: 0064	 New loss: 0.3095, 	Old loss: 0.3095
Embedding size: 0128	 New loss: 0.3102, 	Old loss: 0.3102
Embedding size: 0256	 New loss: 0.3103, 	Old loss: 0.3103
Embedding size: 0512	 New loss: 0.3052, 	Old loss: 0.3052
Embedding size: 1024	 New loss: 0.2986, 	Old loss: 0.2986


In [12]:
num_weights = ((weights ** 2).sum(dim=1)).sum()
norm = ((embedding_size ** 2) * (num_weights ** 2)) - 2 * (embedding_size * (num_weights ** 2)) + (num_weights ** 2)

print(norm)

vTv = ((embedding.transpose(2, 1) @ embedding) ** 2)
vTy = ((embedding.transpose(2, 1) @ assignments) ** 2)
yTy = ((assignments.transpose(2, 1) @ assignments) ** 2)

(vTv.sum() - 2 * vTy.sum() + yTy.sum()) / norm

tensor(305950.6250)


tensor(0.0106)

In [14]:
vTy, yTy, vTv

(tensor([[[131.9500, 151.0961],
          [ 99.2297, 141.4376],
          [108.6486, 132.5084],
          [139.3827, 155.2395],
          [118.7430, 136.2694],
          [130.8699, 145.6270],
          [105.1004, 123.7130],
          [ 77.9538, 135.6935],
          [112.8176, 136.5007],
          [ 73.8812, 133.4724],
          [ 92.0670, 125.5312],
          [ 90.9855,  98.6171],
          [ 98.8411, 139.9132],
          [126.3239, 119.3181],
          [111.6611, 121.3125],
          [119.6591,  99.3209]]]), tensor([[[2304.,    0.],
          [   0., 2704.]]]), tensor([[[49.5241, 28.1220, 24.9829, 30.2371, 25.1409, 30.4873, 25.4457,
           24.9004, 28.0923, 22.4296, 24.6680, 21.7190, 25.5763, 27.3789,
           23.4485, 22.8465],
          [28.1220, 40.7262, 22.3464, 25.1150, 24.0132, 25.2296, 19.4699,
           21.4355, 22.2204, 18.6501, 19.1193, 16.3038, 21.2854, 22.0231,
           18.9963, 22.0369],
          [24.9829, 22.3464, 41.1049, 28.4609, 23.9456, 27.0004, 19.6646,
  

In [4]:
model.layers['mel_projection'].num_mels

-1

In [2]:
from trainer import Trainer, LossFunctions


t = Trainer(output_folder='test/', 
        train_data=None, 
        validation_data=None, 
        model='networks/dpcl_config.json',
        loss_tuples=loss_tuples)

In [3]:
model_dict = torch.load('networks/test.p')
model = model_dict['model']
model.load_state_dict(model_dict['state_dict'])
model.eval()
output = model(data)

t.calculate_loss

<bound method Trainer.calculate_loss of <trainer.Trainer object at 0x7ff20f646ac8>>

In [19]:
LossFunctions['DPCL'].value()

DeepClusteringLoss()

In [70]:
'test'.upper()

'TEST'